## Utility Functions

SimFrame provides various utility functions for inspecting, modifying and saving lattices and lattice elements. This notebook will demonstrate some of this functionality.

For general tips on setting up SimFrame, see [getting_started](./getting_started.ipynb).

In [1]:
import sys
import numpy as np
import matplotlib.pyplot as plt
from yaml import safe_load
from random import uniform

sys.path.insert(0, "../../../../")
import SimulationFramework.Framework as fw  # noqa E402

# Define a new framework instance, in directory 'getting_started'.
#       "clean" will empty (delete everything!) in the directory if true
#       "verbose" will print a progressbar if true
framework = fw.Framework(
        master_lattice='../../../../../masterlattice/MasterLattice/',
        simcodes='../../../../../simcodes/SimCodes',
        directory="./getting_started",
        clean=False,
        verbose=False
    )

scaling = 4

initializing ocelot...


Invalid MIT-MAGIC-COOKIE-1 key

Load the settings from a definitions file and check which codes correspond to each line.

In [2]:
framework.loadSettings("Lattices/clara400_v13_SP3.def")

In [3]:
print([f"{line}: {framework[line].code}" for line in framework.lines])

['generator: ASTRA', 'injector400: astra', 'S02: elegant', 'L02: elegant', 'S03: elegant', 'L03: elegant', 'S04: elegant', 'L4H: elegant', 'S05: elegant', 'VBC: elegant', 'S06: elegant', 'L04: elegant', 'S07: elegant', 'SP3: elegant']


Check the global parameters accessible to this instance of SimFrame and all of the `frameworkLattice` objects.

In [4]:
print(framework.global_parameters)

{'beam': {'filename': None, 'code': None}, 'GPTLICENSE': '', 'delete_tracking_files': False, 'astra_use_wsl': 1, 'master_subdir': '/home/xkc85723/Documents/simframe/docs/source/examples/notebooks/getting_started', 'master_lattice_location': '/home/xkc85723/Documents/masterlattice/MasterLattice/./', 'simcodes_location': '/home/xkc85723/Documents/simcodes/SimCodes/./'}


In [5]:
framework.generator.load_defaults("clara_400_2ps_Gaussian")

In [6]:
print(framework.settings)

FrameworkSettings({'settingsFilename': '/home/xkc85723/Documents/masterlattice/MasterLattice/./Lattices/clara400_v13_SP3.def', 'groups': {'bunch_compressor': {'type': 'chicane', 'elements': ['CLA-VBC-MAG-DIP-01', 'CLA-VBC-MAG-DIP-02', 'CLA-VBC-MAG-DIP-03', 'CLA-VBC-MAG-DIP-04']}}, 'generator': {'code': 'astra', 'default': 'clara_400_3ps'}, 'global': {'ASTRAsettings': {'High_res': True, 'N_red': 1, 'EmitS': True, 'PhaseS': True, 'TrackS': True, 'RefS': True, 'Track_All': True, 'Phase_Scan': False, 'Auto_Phase': True, 'check_ref_part': False, 'Zphase': 1, 'Zemit': 1050, 'H_max': 0.0007, 'H_min': 0.0007, 'title': 'trial', 'run_no': 1, 'Smooth_x': 2, 'Smooth_y': 2, 'Smooth_z': 2, 'Max_scale': 0.1, 'Max_count': 100, 'cell_var': 2, 'min_grid': 3.424657e-13, 'Lmagnetized': False}}, 'elements': {'filename': ['YAML/Injector400.yaml', 'YAML/S02.yaml', 'YAML/L02.yaml', 'YAML/S03.yaml', 'YAML/L03.yaml', 'YAML/S04.yaml', 'YAML/L4H.yaml', 'YAML/S05.yaml', 'YAML/VBC.yaml', 'YAML/S06.yaml', 'YAML/L04.

In [7]:
framework.change_subdirectory('./utility_functions')

### Accessing and modifying elements

Elements and their parameters can be retrieved either using `getElement` or as a key in the `framework` object itself.

In [8]:
print(framework.getElement("CLA-S02-MAG-QUAD-01"))
print(framework["CLA-S02-MAG-QUAD-01"])

objectname='CLA-S02-MAG-QUAD-01' objecttype='quadrupole' objectdefaults={} allowedkeywords=['subelement', 'sub_elements', 'length', 'online_model_name', 'controller_name', 'pv_root', 'pv_suffixes', 'field_integral_coefficients', 'global_rotation', 'centre', 'datum', 'start', 'end', 'buffer_start', 'buffer_end', 'buffer_start_length', 'buffer_end_length', 'position_errors', 'rotation_errors', 'tilt_angle', 'array_names', 'rotation', 'smooth', 'field_reference_position', 'parent', 'k1l', 'k1', 'field_definition', 'gradient', 'fringe_field_coefficient', 'field_type', 'scale_field', 'multipoles'] length=0.1007 centre=[0.0, 0.0, 3.52715] position_errors=[0, 0, 0] rotation_errors=[0, 0, 0] global_rotation=[0.0, 0.0, 0.0] rotation=[0, 0, 0] starting_rotation=0.0 conversion_rules_elegant={'length': 'l', 'entrance_edge_angle': 'e1', 'exit_edge_angle': 'e2', 'edge_field_integral': 'fint', 'half_gap': 'hgap', 'horizontal_size': 'x_max', 'vertical_size': 'y_max', 'output_filename': 'filename', 'cs

In [9]:
print(framework.getElement("CLA-S02-MAG-QUAD-01", "k1l"))
print(framework["CLA-S02-MAG-QUAD-01"].k1l)

-0.128366
-0.128366


All objects of a given type defined in the entire lattice can be accessed as follows.

(NB here cavity on-crest phase is zero and ASTRA convention is followed for the phase sign)

In [10]:
cavnames = [c["name"] for c in framework.getElementType("cavity")]
print({k: v for k, v in zip(cavnames, framework.getElementType("cavity", "phase"))})

{'CLA-HRG1-GUN-CAV-01': 9, 'CLA-L01-LIN-CAV-01': 16, 'CLA-L02-LIN-CAV-01': 23, 'CLA-L03-LIN-CAV-01': 8, 'CLA-L4H-LIN-CAV-01': -184, 'CLA-L04-LIN-CAV-01': -45}


/home/xkc85723/.local/lib/python3.12/site-packages/pydantic/functional_serializers.py:434: UserWarning: Pydantic serializer warnings:
  Expected `float` but got `int64` with value `np.int64(2998500000)` - serialized value may not be as expected
  lambda x, h: h(x), schema=core_schema.any_schema()
/home/xkc85723/.local/lib/python3.12/site-packages/pydantic/functional_serializers.py:434: UserWarning: Pydantic serializer warnings:
  Expected `float` but got `int64` with value `np.int64(11994000000)` - serialized value may not be as expected
  lambda x, h: h(x), schema=core_schema.any_schema()


Various options are available for changing a lattice element, or multiple elements of a given type simultaneously.

In [11]:
# Set all dipole angles in the lattice to zero and use `setElementType`
values = [0 for _ in framework.getElementType("dipole")]
dipnames = [d["name"] for d in framework.getElementType("dipole")]
framework.setElementType("dipole", "angle", values)
print({k: v for k, v in zip(dipnames, framework.getElementType("dipole", "angle"))})

# Set all dipole angles to 0.1 and use modifyElementType
value = 0.1
framework.modifyElementType("dipole", "angle", 0.1)
print({k: v for k, v in zip(dipnames, framework.getElementType("dipole", "angle"))})

{'CLA-VBC-MAG-DIP-01': 0.0, 'CLA-VBC-MAG-DIP-02': 0.0, 'CLA-VBC-MAG-DIP-03': 0.0, 'CLA-VBC-MAG-DIP-04': 0.0, 'CLA-SP3-MAG-DIP-01': 0.0}
{'CLA-VBC-MAG-DIP-01': 0.1, 'CLA-VBC-MAG-DIP-02': 0.1, 'CLA-VBC-MAG-DIP-03': 0.1, 'CLA-VBC-MAG-DIP-04': 0.1, 'CLA-SP3-MAG-DIP-01': 0.1}


In [12]:
s02q1 = "CLA-S02-MAG-QUAD-01"
s02q2 = "CLA-S02-MAG-QUAD-02"
setattr(framework.elementObjects[s02q1], 'k1l', 1)
print(f"{s02q1} k1l = {framework.getElement(s02q1, 'k1l')}")
framework[s02q1].k1l = 2
print(f"{s02q1} k1l = {framework.getElement(s02q1, 'k1l')}")
framework.modifyElement(s02q1, "k1l", 3)
print(f"{s02q1} k1l = {framework.getElement(s02q1, 'k1l')}")
framework.modifyElement(s02q1, ["k1l", "n_kicks"], [3, 5])
print(f"{s02q1} k1l = {framework.getElement(s02q1, 'k1l')}, n_kicks = {framework[s02q1].n_kicks}")
framework.modifyElement(s02q1, {"k1l": 4, "n_kicks": 6})
print(f"{s02q1} k1l = {framework.getElement(s02q1, 'k1l')}, n_kicks = {framework[s02q1].n_kicks}")
framework.modifyElements([s02q1, s02q2], {"k1l": 5, "n_kicks": 7})
print(f"{s02q1} k1l = {framework.getElement(s02q1, 'k1l')}, n_kicks = {framework[s02q1].n_kicks}, \
{s02q2} k1l = {framework.getElement(s02q2, 'k1l')}, n_kicks = {framework[s02q2].n_kicks}")

CLA-S02-MAG-QUAD-01 k1l = 1.0
CLA-S02-MAG-QUAD-01 k1l = 2.0
CLA-S02-MAG-QUAD-01 k1l = 3.0
CLA-S02-MAG-QUAD-01 k1l = 3.0, n_kicks = 5
CLA-S02-MAG-QUAD-01 k1l = 4.0, n_kicks = 6
CLA-S02-MAG-QUAD-01 k1l = 5.0, n_kicks = 7, CLA-S02-MAG-QUAD-02 k1l = 5.0, n_kicks = 7


Save the specific updates to these parameters

In [13]:
framework.saveParametersFile("params.yaml", framework.getElementType("dipole"))

Save a file containing these changes

In [14]:
framework.save_changes_file("changes.yaml", typ=None, elements=None, dictionary=False)

Modify the changes file and apply it to the framework, then check that the modified changes were applied

Alternatively, you can use `framework.load_changes_file(filename)`

In [15]:
with open("changes.yaml", "r") as f:
    changes = safe_load(f)

for name, values in changes.items():
    for param in values.keys():
        changes[name][param] = 0

framework.apply_changes(changes, verbose=True)

for name, values in changes.items():
    for param in values.keys():
        print(f"{name} {framework.getElement(name, param)}")

modifying  CLA-S02-MAG-QUAD-01 [ k1l ]  =  0
modifying  CLA-S02-MAG-QUAD-01 [ n_kicks ]  =  0
modifying  CLA-S02-MAG-QUAD-02 [ k1l ]  =  0
modifying  CLA-S02-MAG-QUAD-02 [ n_kicks ]  =  0
modifying  CLA-VBC-MAG-DIP-01 [ angle ]  =  0
modifying  CLA-VBC-MAG-DIP-02 [ angle ]  =  0
modifying  CLA-VBC-MAG-DIP-03 [ angle ]  =  0
modifying  CLA-VBC-MAG-DIP-04 [ angle ]  =  0
modifying  CLA-SP3-MAG-DIP-01 [ angle ]  =  0
CLA-S02-MAG-QUAD-01 0.0
CLA-S02-MAG-QUAD-01 0
CLA-S02-MAG-QUAD-02 0.0
CLA-S02-MAG-QUAD-02 0
CLA-VBC-MAG-DIP-01 0.0
CLA-VBC-MAG-DIP-02 0.0
CLA-VBC-MAG-DIP-03 0.0
CLA-VBC-MAG-DIP-04 0.0
CLA-SP3-MAG-DIP-01 0.0
